In [1]:

from pathlib import Path 
import numpy as np
import tifffile as tifff
import os
import json
from PIL import Image, ImageDraw
#from skimage.draw import polygon2mask

In [2]:
def polygon2mask(xysize, polygon_outline):
    mask = Image.new('L', xysize, 0)
    ImageDraw.Draw(mask).polygon(polygon_outline, outline=1, fill=1)
    return np.array(mask)
xysize=(1000,1000)

In [3]:
# xysize=(1000,1000)

In [4]:
base_path = '/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/artifact_bone_masks/GeoJSON files_artifact bone'

In [5]:
def process_geojson(geojson_file, xysize=(1000, 1000)):
    with open(geojson_file) as f:
        data = json.load(f)
    
    mask_combined = np.zeros(xysize, dtype=np.uint8)
    non_bone_mask = np.zeros(xysize, dtype=np.uint8)
    
    for feature in data['features']:
        geometry = feature['geometry']
        if 'name' in feature['properties'] and feature['properties']['name'].startswith('Bone'):
            if geometry['type'] == 'Polygon':
                for polygon_coords in geometry['coordinates']:
                    polygon_outline = [(coord[0], coord[1]) for coord in polygon_coords]
                    mask = polygon2mask(xysize, polygon_outline)
                    mask_combined = np.logical_or(mask_combined, mask)
            elif geometry['type'] == 'MultiPolygon':
                for polygon in geometry['coordinates']:
                    for polygon_coords in polygon:
                        polygon_outline = [(coord[0], coord[1]) for coord in polygon_coords]
                        mask = polygon2mask(xysize, polygon_outline)
                        mask_combined = np.logical_or(mask_combined, mask)
            else:
                print(f"Unknown geometry type: {geometry['type']}")
        else:
            if geometry['type'] == 'Polygon':
                for polygon_coords in geometry['coordinates']:
                    polygon_outline = [(coord[0], coord[1]) for coord in polygon_coords]
                    mask = polygon2mask(xysize, polygon_outline)
                    non_bone_mask = np.logical_or(non_bone_mask, mask)
            elif geometry['type'] == 'MultiPolygon':
                for polygon in geometry['coordinates']:
                    for polygon_coords in polygon:
                        polygon_outline = [(coord[0], coord[1]) for coord in polygon_coords]
                        mask = polygon2mask(xysize, polygon_outline)
                        non_bone_mask = np.logical_or(non_bone_mask, mask)
            else:
                print(f"Unknown geometry type: {geometry['type']}")
    mask_combined = np.logical_and(mask_combined, np.logical_not(non_bone_mask))
    return (mask_combined * 255).astype(np.uint8)

In [9]:
for file in os.listdir(base_path):
    sample = file.split('.')[0]
    if file.endswith('.geojson'):
        print(f"Processing {file}")
        mask = process_geojson(os.path.join(base_path, file))
        output_file = os.path.join('/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/artifact_bone_masks/bone_masks', sample + '_bone_masks.tiff')
        Image.fromarray(mask).save(output_file)

Processing TS-373_IMC05_MGUS_001.geojson
Processing TS-373_IMC93_B_002.geojson
Processing TS-373_IMC84_B_002.geojson
Processing TS-373_IMC04_MGUS_002.geojson
Processing TS-373_IMC89_B_001.geojson
Processing TS-373_IMC50_B_001.geojson
Processing TS-373_IMC89_B_002.geojson
Processing TS-373_IMC04_MGUS_001.geojson
Processing TS-373_IMC93_B_001.geojson
Processing TS-373_IMC37-1_MGUS_001.geojson
Processing TS-373_IMC08_UB_002.geojson
Processing TS-373_IMC88_B_002.geojson
Processing TS-373_IMC66_B_001.geojson
Processing TS-373_IMC92_B_001.geojson
Processing TS-373_IMC38_B_001.geojson
Processing TS-373_IMC45_B_002.geojson
Processing TS-373_IMC18-1_MGUS_001.geojson
Processing TS-373_IMC67-1_B_001.geojson
Processing TS-373_IMC85_UB_002.geojson
Processing TS-373_IMC90.1_B_002.geojson
Processing TS-373_IMC12_UB_001.geojson
Processing TS-373_IMC15_B_002.geojson
Processing TS-373_IMC86_B_002.geojson
Processing TS-373_IMC87-1_UB_002.geojson
Processing TS-373_IMC72_MGUS_002.geojson
Processing TS-373_